In [1]:
#DGP
import numpy as np
import pandas as pd
import random

# n = 200  #firm num
# T = 100  #time num
#To simplify the simulation, we change the x_{i,(t-1)} ,r_{i,t} to x_n, r_n, 
# which means denote all the variable with  n th news rather than i th firm in t th day
np.random.seed(66)
S = 10000  #word num
n = 500   #news num
p = 2  #dim of latent space

#x ~ Normal
mu_x = 0.03
sigma_x = 0.05
x = np.random.normal(mu_x,sigma_x,(n,p))  #latent variable n x p

# beta controls signal-noise ratio 0.5, 10, 100 ###normal?
beta = np.random.normal(1,0,p) 
#noise~ Normal
sigma_epsilon = 0.05
epsilon = np.random.normal(0,sigma_epsilon,n)

# return
r = x @ beta + epsilon 
ratio_r = np.var(x @ beta)/(np.var(x @ beta)+np.var(epsilon))

#Gamma controls sentiment signals S x p
K = 10 #num of neg/pos sentiment words nums
s1 = 5 #lower bound of the sentiment strength  (non-neg)
s2 = 15 #upper bound of the sentiment strength 
gamma = np.zeros((S,p))  
gamma[:K] = np.random.uniform(s1,s2,(K,p)) 
gamma[-K:] = np.random.uniform(-s2,-s1,(K,p))

#theta control the sparsity of word matrix W
mu_b = 0.5
mu_theta = -10
sigma_theta = 0.05
theta = np.random.binomial(1, mu_b,(n,S)) * np.random.normal(mu_theta, sigma_theta,(n,S))

#paramenter of Poisson distribution
lam = np.exp( x @ gamma.T + theta)

# words count matrix
W = np.random.poisson(lam,(n,S))


# words count of each article
total_words_example = np.mean(np.sum(W,axis=1))

## calculate sparcity 
sparcity = 1 - np.count_nonzero(W)/W.size

print("sparsity of W:", sparcity)
print("signal-to-noise ratio:", ratio_r)
print("neg return num:",np.sum(r<0) )



sparsity of W: 0.6837092
signal-to-noise ratio: 0.6590253129256418
neg return num: 123


In [66]:
'''

'''
import numpy as np
import pandas as pd
from scipy import sparse
import os

import heapq
from sklearn.linear_model import Lasso ###导入Lasso回归算法
from sklearn.decomposition import PCA    
from sklearn import linear_model
from sklearn.decomposition import TruncatedSVD 

################## Could be changed part #######################
choose_return = 'risk' # Risk adjusted return or Market adjusted return
return_folder = 'daily_single_final/' # the folder of files with both return and news merged together
word_file = 'first1w'
screen_num = 1000 # screening number
variable_num = 100 # final words want to choose
choose_startyear = '2013'
jobs = 30

####################
def Farmpredict(n,K,p,r,W_train,freqnum=10000):
    '''
    Input:
        n: Number of news, int
        K: Number of words, we choose 10,000 here, int
        p: How many PCA Factors want to choose, according Fan(2021), int
        r: stock return vector, n*1, array
        W_train: sparse matrix of total dataset, dimension in n * K, array
    
    Return:
        S_set: correlation screening results, list in length of 1000
        lasso_index: Non-zero index of optimal lasso model of S_set, list
        lasso_Beta: optimal lasso model's coefficients, array
        lasso_beta: Non-zero coefficients of optimal lasso model
        loading: In-Sample B_hat
        insregLRcoef: In-Sample linear regression coefficients between r and pcafactor
        lasso_alpha: optimal lasso model's intercept
        insregLRint: In-Sample linear regression intercept between r and pcafactor
    '''
    # PCA for words count matrix
    svdtest = TruncatedSVD(n_components=p, n_iter=7, random_state=42)
    # First screen
    Wsum = np.sum(W_train, axis=0)
    freq_index = heapq.nlargest(freqnum, enumerate(Wsum), key=lambda x: x[1])
    W_train = W_train[freq_index]
    print('W_train first screen done')
    
    pcafactor = svdtest.fit_transform(W_train) 
    pcau = np.array(W_train - (pcafactor @ svdtest.components_))
    loading = svdtest.components_.T  # output In-Sample B_hat
    print('pca done')    
    # linear regression of return on estimated factor
    # r ~ a_1 + regcoef*pcafactor + r_u
    
    # initial parameters
    regLR = linear_model.LinearRegression(n_jobs=jobs)

    # fit model 
    regLR.fit(pcafactor,r)
    yu = r - regLR.intercept_ - pcafactor @ regLR.coef_ 
    
    insregLRint = regLR.intercept_  # output In-Sample linear regression intercept
    insregLRcoef = regLR.coef_   # output In-Sample linear regression coefficients

    # calculate correlation (r_u, pcau)
    corrvector = []
    for s in range(0, K):   
        corrvector.append(abs(np.corrcoef(yu,pcau[:,s])[0,1]))
    
    max_number = heapq.nlargest(screen_num, corrvector) 
    max_index = []
    for t in max_number:
        index = corrvector.index(t)
        max_index.append(index)
        corrvector[index] = 0

    S_set = max_index
    
    # lasso fitting
    # screening set
    pcaus = pcau[:,S_set]

    # linear regression of idiosyncratic components on estimated factor
    # pcaus ~ a_2 + regcoef*pcafactor + x_tilde

    # initial parameters
    regLR_u = linear_model.LinearRegression(n_jobs=jobs)
    
    # fit model 
    xtilde = np.zeros((n,screen_num))
    for s in range(screen_num):
        regLR_u.fit(pcafactor,pcaus[:,s])
        xtilde[:,s] = pcaus[:,s] - regLR_u.intercept_ - pcafactor @ regLR_u.coef_ 

    lambdas = 10 ** np.linspace(-4,-2,num=200)
    l_num = len(lambdas)
    pred_num = screen_num
    
    # prepare data for enumerate
    coeff_num = np.zeros(l_num)
    coeff_beta = np.zeros((l_num, pred_num))
    # enumerate through lambdas with index and i
    for ind, i in enumerate(lambdas):    
        reg = Lasso(alpha = i)
        reg.fit(xtilde, yu)
    
        coeff_beta[ind,:] = reg.coef_
        coeff_num[ind] = len(np.nonzero(reg.coef_)[0])
     
    df_lam = pd.DataFrame(coeff_num, columns=['number of variables'])    
    df_lam['lambda'] = (lambdas)
    # returns the lambda value where number of variables has number of 20 or near 20.
    best_lambda = df_lam['lambda'][find_nearest(df_lam['number of variables'],variable_num)]

    # build up optimal model 
    # r_u ~ alpha + x_tilde*beta
    lasso_best = Lasso(alpha=best_lambda)
    lasso_best.fit(xtilde,yu)
        
    # get optimal model's coefficients 
    lasso_alpha = lasso_best.intercept_
    lasso_Beta = lasso_best.coef_

    lasso_index0 =  list(np.nonzero(lasso_Beta)[0])
    lasso_index = [S_set[x] for x in lasso_index0]
    
    lasso_beta = lasso_Beta[np.nonzero(lasso_Beta)[0]]

    print(lasso_index)
    
    return S_set,lasso_index,lasso_Beta,lasso_beta,loading,insregLRcoef,lasso_alpha,insregLRint    